# Basic connectivity variables.
Only change them if values of lab are modified.

In [2]:
# Basic connectivity variables.
# Only change if values of lab are modified.
router = "192.168.10.4"
port = 830
username = "vagrant"
password = "vagrant"
protocol = "ssh"

In [3]:
from ydk.providers import CodecServiceProvider, NetconfServiceProvider
from ydk.services import CRUDService, CodecService
# We need to create a provider to connect with the router. 
# There is a considerable theory behind this step (netconf, restconfg, etc.), 
# but we'll leave the details aside for this practice.
provider = NetconfServiceProvider(address=router,
                                      port=port,
                                      username=username,
                                      password=password, 
                                      protocol=protocol)
crud = CRUDService()

# YDK "binds" data from/to the router into python objects. 
# Still, we might want to look at the responses in xml for a quick. The next objects allow us to do that.

cd_provider = CodecServiceProvider(type="xml")
codec = CodecService()

In [ ]:
# In case you want verbose logging, uncomment this lines.
#import logging
#log = logging.getLogger('ydk')
#log.setLevel(logging.DEBUG)
#ch = logging.StreamHandler()
#log.addHandler(ch)

# BGP Configuration
We'll have a twist in this task and we will be using one of the openconfig modules: openconfig_bgp (https://github.com/openconfig/public/blob/master/release/models/bgp/openconfig-bgp.yang)

Note: We could probably use one of the XR models: Cisco_IOS_XR_ipv4_bgp_cfg and Cisco_IOS_XR_ipv4_bgp_datatypes . Try them out if you want, it should lead to the same result.

In [4]:
from ydk.models.openconfig import openconfig_bgp \
    as oc_bgp
from ydk.models.openconfig import openconfig_bgp_types as oc_bgp_types
from ydk.types import Empty

As with the other exercises, you can configure the router first, read the configuration and learn how to fill the objects using it.

In [12]:
bgp = xr_ipv4_bgp_cfg.Bgp()  # create object
# read data from NETCONF device. This will be empty if you did not configure BGP.
bgp_read = crud.read(provider, bgp)
#print(codec.encode(cd_provider, bgp_read))

Now, try to configure BGP on the router using objects. If you want to make it compatible with the quagga router, please use values that match its configuration (ASN, source interface, etc.)

A potential solution for this excesise is down. Please note the the solution requires you to have configured the Loopback in the router.

In [ ]:
# Your solution goes here
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#

In [21]:
bgp = oc_bgp.Bgp()  # create object
bgp.global_.config.as_ = 65001
bgp.global_.config.router_id = "1.1.1.3"
afi_safi = bgp.global_.afi_safis.AfiSafi()
afi_safi.afi_safi_name = oc_bgp_types.Ipv4UnicastIdentity()
afi_safi.config.afi_safi_name = oc_bgp_types.Ipv4UnicastIdentity()
afi_safi.config.enabled = True
bgp.global_.afi_safis.afi_safi.append(afi_safi)

# configure IBGP neighbor
neighbor = bgp.neighbors.Neighbor()
neighbor.neighbor_address = "1.1.1.1"
neighbor.config.neighbor_address = "1.1.1.1"
neighbor.config.peer_as = 65001
neighbor.transport.config.local_address = "Loopback0"
afi_safi = neighbor.afi_safis.AfiSafi()
afi_safi.afi_safi_name = oc_bgp_types.Ipv4UnicastIdentity()
afi_safi.config.afi_safi_name = oc_bgp_types.Ipv4UnicastIdentity()
afi_safi.config.enabled = True
#afi_safi.apply_policy.config.export_policy.append("POLICY2")
neighbor.afi_safis.afi_safi.append(afi_safi)

bgp.neighbors.neighbor.append(neighbor)

In [ ]:
crud.create(provider, bgp)